<a href="https://colab.research.google.com/github/sugiyama404/BuckTest/blob/main/buck_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
 
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
!pip install backtesting

--2021-06-21 07:28:08--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-06-21 07:28:09--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-06-21 07:28:09--  https://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving nchc.dl.sourceforge.ne

In [ ]:
import pandas_datareader.data as web
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

import pandas as pd
from google.colab import drive

In [ ]:
start = datetime.date(2010,1,1)
end = datetime.date.today()
df = web.DataReader('SPY', 'yahoo', start, end) #データの取得


drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
work_dir = 'Colab Notebooks/workspace/export/'
sp500_path = '/content/drive/My Drive/' + nov_dir + 'sp500_candle.csv'
cpi_path = '/content/drive/My Drive/' + nov_dir + 'corecoreCPI.csv'
save_path = '/content/drive/My Drive/' + work_dir

df.to_csv(sp500_path)
print(df.shape)
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
(2885, 6)


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,113.389999,111.510002,112.370003,113.330002,118944600.0,90.790062
2010-01-05,113.680000,112.849998,113.260002,113.629997,111579900.0,91.030411
2010-01-06,113.989998,113.430000,113.519997,113.709999,116074400.0,91.094490
2010-01-07,114.330002,113.180000,113.500000,114.190002,131091100.0,91.479050
2010-01-08,114.620003,113.660004,113.889999,114.570000,126402800.0,91.783432


In [ ]:
class SmaCross(Strategy):
    n1 = 10 # 短期SMA
    n2 = 30 # 長期SMA

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self): # チャートデータの行ごとに呼び出される
        if crossover(self.sma1, self.sma2): # sma1がsma2を上回った時
            self.buy() # 買い
        elif crossover(self.sma2, self.sma1):
            self.position.close() # 売り

# バックテストを設定
bt = Backtest(
    df, # チャートデータ
    SmaCross, # 売買戦略
    cash=1000, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ
)

In [ ]:
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
bt.plot() # 実行結果（グラフ）

Start                     2010-01-04 00:00:00
End                       2021-06-18 00:00:00
Duration                   4183 days 00:00:00
Exposure Time [%]                     69.3588
Equity Final [$]                      1369.59
Equity Peak [$]                       1401.56
Return [%]                            36.9593
Buy & Hold Return [%]                 266.117
Return (Ann.) [%]                     2.78531
Volatility (Ann.) [%]                 10.1241
Sharpe Ratio                         0.275117
Sortino Ratio                        0.379819
Calmar Ratio                         0.145118
Max. Drawdown [%]                    -19.1934
Avg. Drawdown [%]                    -2.05164
Max. Drawdown Duration     1627 days 00:00:00
Avg. Drawdown Duration      102 days 00:00:00
# Trades                                   47
Win Rate [%]                          46.8085
Best Trade [%]                        13.7881
Worst Trade [%]                      -6.31925
Avg. Trade [%]                    

# 最適化

In [ ]:
output2=bt.optimize(n1=range(10, 70, 5), n2=range(10, 70, 5), maximize='Equity Final [$]')
print(output2)
bt.plot()

Start                     2010-01-04 00:00:00
End                       2021-06-18 00:00:00
Duration                   4183 days 00:00:00
Exposure Time [%]                     72.0277
Equity Final [$]                      2421.01
Equity Peak [$]                       2437.46
Return [%]                            142.101
Buy & Hold Return [%]                 266.117
Return (Ann.) [%]                     8.02927
Volatility (Ann.) [%]                 11.4783
Sharpe Ratio                          0.69952
Sortino Ratio                         1.05383
Calmar Ratio                         0.389333
Max. Drawdown [%]                    -20.6231
Avg. Drawdown [%]                    -1.65894
Max. Drawdown Duration     1169 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   22
Win Rate [%]                          54.5455
Best Trade [%]                        19.8868
Worst Trade [%]                      -5.96478
Avg. Trade [%]                    

In [ ]:
class myCustomStrategy(Strategy):
    def init(self):
        pass

    def next(self): 
        self.buy if self.data.Close[-1]> self.data.Open[-1] else self.sell()

In [ ]:
bt = Backtest(df, myCustomStrategy, cash=100000, commission=.00004)
bt.run()

Start                     2010-01-04 00:00:00
End                       2021-06-18 00:00:00
Duration                   4183 days 00:00:00
Exposure Time [%]                     42.3917
Equity Final [$]                            0
Equity Peak [$]                        111705
Return [%]                               -100
Buy & Hold Return [%]                 266.117
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                  185.48
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                        -100
Avg. Drawdown [%]                    -15.8828
Max. Drawdown Duration     4004 days 00:00:00
Avg. Drawdown Duration      522 days 00:00:00
# Trades                                    5
Win Rate [%]                                0
Best Trade [%]                       -80.3617
Worst Trade [%]                      -99.2809
Avg. Trade [%]                    

In [ ]:
bt.plot()

In [ ]:
import talib as ta

def MACD(close, n1, n2, ns):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=ns)
    return macd, macdsignal

In [ ]:
class MACDCross(Strategy):
    n1 = 12 #短期EMAの期間
    n2 = 26 #長期EMAの期間
    ns = 9 #シグナル（MACDのSMA）の期間

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, self.data.Close, self.n1, self.n2, self.ns)

    def next(self): # チャートデータの行ごとに呼び出される
        if crossover(self.macd, self.macdsignal): #macdがsignalを上回った時
            self.buy() # 買い
        elif crossover(self.macdsignal, self.macd): #signalがmacdを上回った時
            self.position.close() # 売り

In [ ]:
# バックテストを設定
bt = Backtest(
    df, # チャートデータ
    MACDCross, # 売買戦略
    cash=1000, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
bt.plot() # 実行結果（グラフ）

Start                     2010-01-04 00:00:00
End                       2021-06-18 00:00:00
Duration                   4183 days 00:00:00
Exposure Time [%]                     53.7262
Equity Final [$]                      1094.61
Equity Peak [$]                       1116.19
Return [%]                            9.46082
Buy & Hold Return [%]                 266.117
Return (Ann.) [%]                    0.792724
Volatility (Ann.) [%]                 8.48867
Sharpe Ratio                        0.0933861
Sortino Ratio                        0.130318
Calmar Ratio                        0.0438188
Max. Drawdown [%]                    -18.0909
Avg. Drawdown [%]                     -3.3052
Max. Drawdown Duration     2016 days 00:00:00
Avg. Drawdown Duration      254 days 00:00:00
# Trades                                  121
Win Rate [%]                          42.1488
Best Trade [%]                        9.87255
Worst Trade [%]                      -4.89152
Avg. Trade [%]                    

In [ ]:
#最適化
output2=bt.optimize(n1=range(2, 100, 10),n2=range(2, 300, 20),ns=range(2, 50, 5), maximize='Equity Final [$]', max_tries=200)
print(output2)
bt.plot()

Start                     2010-01-04 00:00:00
End                       2021-06-18 00:00:00
Duration                   4183 days 00:00:00
Exposure Time [%]                     53.4142
Equity Final [$]                      1591.39
Equity Peak [$]                       1638.53
Return [%]                            59.1387
Buy & Hold Return [%]                 266.117
Return (Ann.) [%]                     4.14173
Volatility (Ann.) [%]                 8.96572
Sharpe Ratio                         0.461952
Sortino Ratio                        0.661593
Calmar Ratio                         0.245268
Max. Drawdown [%]                    -16.8865
Avg. Drawdown [%]                    -1.65025
Max. Drawdown Duration     2348 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                                   29
Win Rate [%]                          44.8276
Best Trade [%]                        15.5035
Worst Trade [%]                      -2.71477
Avg. Trade [%]                    